In [6]:
!pip install ucimlrepo
!pip install sentence_transformers
!pip install umap-learn

SyntaxError: invalid syntax (2615332520.py, line 1)

### Load Data

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
online_retail = fetch_ucirepo(id=352) 
  
# data (as pandas dataframes) 
X = online_retail.data.features 
X

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


### Clean Data

In [22]:
X_clean = X[X.Description.apply(lambda x:isinstance(x,str) and len(x)>1)]
X_clean['Description'] = X_clean.Description.apply(lambda x:x.replace("'","").replace('.','').strip()).values
X_clean = X_clean.reset_index(drop=True)
unique_rows = X_clean.drop_duplicates('Description').index.values
X_unique = X_clean.iloc[unique_rows]
len(X_clean),len(X),len(unique_rows)

/tmp/ipykernel_6768/887559805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_clean['Description'] = X_clean.Description.apply(lambda x:x.replace("'","").replace('.','').strip()).values


(540408, 541909, 4208)

### Create Embeddings

In [23]:
from sentence_transformers import SentenceTransformer
# best model according to https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer("all-mpnet-base-v2")


# Sentences are encoded by calling model.encode()
embeddings = model.encode(X_unique.Description.values,batch_size=64)


### Lower dimensions

In [24]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

### Save Data

In [26]:
import pickle
with open("umap_embeddings.pkl","wb") as fd:
    pickle.dump(umap_embeddings,fd)
X_clean.to_csv("data_full.csv",index=False)
X_unique.to_csv("data_unique.csv",index=False)